# Final Course Project

<b>Guan GUAN | GGS8662
<br><b>Jingyao YU | JYB4658
<br><b>Lingfei CUI | LCY6897

<h2>Background</h2>

Nowadays, people download Android Apps from Google Play and IOS Apps from Apple Store. In the Google Play, people feel it’s difficult to find the best one among those similar Apps especially for some new ones. It is impossible for them to use each of them and find the most usable one. So a more precisely rating is great and convenient to give users a reference when choose a functional App. An accurate rate could save time and give people better experience. Also it could help developers to improve their product.

<br>Our task is helpful that people could get basic idea about an app that they are not familiar with. They could basically know which app is most likely good among their multiple choices. Meanwhile, developers could use this rating as a reference of prediction before releasing updated version. For example, they may have an idea of a more popular name of their app.

<h2>Dataset and Methodologies</h2>

In this project, we investigated and predicted the rating of Apps based on attributes of the 12 terms, includes App, Category, Reviews, Size, Installs, Type, Price, Content Rating, Genres, Last Updated, Current Ver and Android Ver. We scratch data from www.kaggle.com and use the decision tree and LOOCV to get our preliminary results.

In [58]:
class Node:
    def __init__(self):
        self.label = None
        self.children = {}
        self.saveChildren={}
        self.isLeaf = True
        self.attribute = ''
        # you may want to add additional fields here...

    def __init__(self, attribute, children, label, isLeaf=False):
        self.attribute = attribute
        self.children = children
        self.label = label
        self.isLeaf = isLeaf


    def get_label(self):
        return self.label

    def get_children(self):
        return self.children

    def get_child(self, val):
        if self.children.get(val):
            return self.children[val]
        return None

    def get_atrribute(self):
        return self.attribute

    def checkLeaf(self):
        return self.isLeaf

    def set_to_leaf(self,label):
        self.saveChildren=self.children
        self.children={}
        self.isLeaf=True
        self.label=label

    def recoveprune(self):
        self.children = self.saveChildren
        self.saveChildren = {}
        self.isLeaf = False
        self.label = None


Since we used the decision tree so the first and probably the trickiest work we need to do is to design our tree nodes. We found it is hard to decide what features need to be used to best describe the node, so firstly followed the basic logic of a tree which is a tree could not exist without children, we decided to include children to clarify the child node of the root node; then we knew that we could get the final classification at the leaf nodes of our tree, the feature isLeaf is used as the symbol as the leaf nodes. Together with the isLeaf feature, the classification of the input app could be presented by leaf nodes’ label, then label feature is added as part of the design of the tree node; at last, the process of the classification is the process of examining whether the input app satisfying various attributes, then the attribute of the node is also important as it is the judging criteria of the input app, therefore attribute is also part of the features of the tree nodes.

<br>Above the discussion, the node for our tree is fully designed. There are four features in the node, which are attribute, children, label and isLeaf; attribute describes what the node it is, since the raw data that we used maintains 13 attributes, we need to clarify the attribute of every node; children is just the normal children for a tree which describes the child nodes of the tree; label feature only work when you reach the leaf nodes which indicates the final category of the input app; and isLeaf acts like the notice to tell whether the node is a leaf node, and this feature should work with the label feature together.

In [59]:
from collections import Counter
import copy
import math

def ID3(examples, default):
    '''
    Takes in an array of examples, and returns a tree (an instance of Node)
    trained on the examples.  Each example is a dictionary of attribute:value pairs,
    and the target class variable is a special attribute with the name "Class".
    Any missing attributes are denoted with a value of "?"
    '''
    completeData = missingProcess(examples)
    classify_values = [data['Class'] for data in completeData]

    if len(completeData) == 0:
        return Node('Class', {}, default, True)

    # only have one classfication group
    elif classify_values.count(classify_values[0]) == len(classify_values):
        return Node('Class', {}, classify_values[0], True)

    # only have one attribute
    elif len(completeData[0]) == 1:
        return Node('Class', {}, mostCommon(completeData, 'Class'), True)

    else:
        # this is the best attribute, attribute can not be the 'Class'
        best = chooseAttr(completeData)  
        root = Node(best, {}, None, False)

        for value in bestAttrValues(completeData, best):
            newData = getExamplesWithValue(completeData, best, value)
            subtree = ID3(newData, mostCommon(newData, 'Class'))
            root.children[value] = subtree
    return root


# according to the gainfeature, find attr which has the maximum gainfeature
def chooseAttr(examples):
    max = 0
    max_attr = ''
    for key in examples[0].keys():

        if key != 'Class':
            gain = gainfeature(examples, key)
            if max <= gain:
                max = gain
                max_attr = key

    return max_attr


def test(node, examples):
    '''
    Takes in a trained tree and a test set of examples.  Returns the accuracy (fraction
    of examples the tree classifies correctly).
    '''
    examples = missingProcess(examples)
    correct = 0.0
    start = (int)(len(examples) / 10 * 9)
    end = len(examples) - 1
    for i in examples[start : end]:
        if evaluate(node, i) == i['Class']:
            correct += 1.0

    result = correct / (end - start + 1)
    return result

def evaluate(node, example):
    '''
    Takes in a tree and one example.  Returns the Class value that the tree
    assigns to the example.
    '''
    if node == None:
        return example['Class']
    if node.get_children() == {}:
        result = node.get_label()
        return result
    return evaluate(node.get_child(example[node.get_atrribute()]), example)


def missingProcess(examples):
    '''
    For each attribute, find the most common value for this
    :param example: [dict(a=1, b=0, Class=1),
                     dict(a=1, b=0, Class=1)
                     dict(a=0, b=1, Class=1)]
    :return: dict(a=1, b=0)
    '''
    commonDic = {}
    for key in examples[0].keys():
        if key != "Class":
            commonDic[key] = mostCommon(examples, key)

    # update the examples again
    for example in examples:
        for key in example.keys():
            if example[key] == 'NaN':
                example[key] = commonDic[key]

    return examples


def mostCommon(examples, key):
    valueDic = {}
    values = [example[key] for example in examples]
    # for each key, make a dict to store all their value and its frequence
    for v in values:
        if v in valueDic:
            valueDic[v] += 1.0
        else:
            valueDic[v] = 1.0

    # return the max frequence's value
    max_value = ''
    max = 0.0
    for value in valueDic.keys():
        if max < valueDic.get(value):
            max = valueDic[value]
            max_value = value
    return max_value


def entropy(examples):
    data = [example['Class'] for example in examples]
    index = {}
    for i in data:
        if i not in index:
            index[i] = 1
        else:
            index[i] += 1
    result = 0.0
    for j in index:
        result += index[j] / len(data) * (math.log2(index[j] / len(data)))
    return result * -1

def gainfeature(examples,attri):
    data = [example[attri] for example in examples]
    index = {}
    for i in data:
        if i not in index:
            index[i] = 1
        else:
            index[i] += 1
    result = 0.0
    for j in index:
        testexample = [example for example in examples if example[attri] == j]
        result += index[j] / len(data) * entropy(testexample)
    result = entropy(examples) - result
    return result

def getExamplesWithValue(data, attribute, value):
    '''
    Takes in a complete set of examples, the target attribute and the value
    find all examples that the attribute' field is equal to value
    :param data:
    :param attribute:
    :param value:
    :return: array of dictionary
    '''
    res = []
    for example in data:
        if example[attribute] == value:
            newExample = {}
            for key, val in example.items():
                if key != attribute:
                    newExample[key] = val
            res.append(newExample)
    return res

# find all different values of attribute in examples
def bestAttrValues(examples, attribute):
    '''
    Takes in a complete set of examples and the target attribute, find all the unique
    values in the attribute
    :param examples: list of dictionary
    :param attribute:
    :return: a list of values
    '''
    res = []
    for data in examples:
        if data[attribute] not in res:
            res.append(data[attribute])

    return res


def prune(node, examples):
    '''
    Takes in a trained tree and a validation set of examples.  Prunes nodes in order
    to improve accuracy on the validation data; the precise pruning strategy is up to you.
    '''
    return pruneWithroot(node, examples, node)

def pruneWithroot(node, examples, root):
    if node.isLeaf == True:
        return node

    if isChildAllLeaf(node):
        currentScore = test(root, examples)
        label = mostFreLabel(node)
        node.set_to_leaf(label)
        AfterScore = test(root, examples)
        if currentScore > AfterScore:
            node.recoveprune()
        return node
    else:
        for child in node.children:
            evaluateNode = pruneWithroot(node.children[child], examples, root)
            node.children[child] = evaluateNode
        return node

def isChildAllLeaf(node):
    '''
    for the given node, check the node position, is the position of node could be pruned?
    :param node:
    :return: True or False
    '''
    test_children = node.get_children()
    for child in test_children.values():
        if not child.isLeaf:
            return False
    return True

def mostFreLabel(node):
    labelList = [child.get_label() for child in node.get_children().values()]
    mostfrelabel = Counter(labelList).most_common(1)
    return mostfrelabel[0][0]

We used ID3 algorithm to do the decision tree learning, but before the algorithm starts to work, we did the data pre-processing since there are some data losses in the raw data from Kaggle, here you can see that we firstly input our raw data into the missingProcess function, and basically the usage of this function is to make up the loss attribute with the attribute which repeated most often. Then we went through the three basic corner cases, which takes the bounding cases into consideration. By using the function called chooseAttr(choose attribute), we could set the attribute of every root node according to the value of the gainfeature, and basically the gainfeature is computed by calculating the entropy and then the information gain of every attribute, the function selects the attribute which maintains the smallest entropy (or we can say the largest information gain) step by step in each iteration.

In [60]:
import csv
import codecs

def parse(filename):
    '''
    takes a filename and returns attribute information and all the data in array of dictionaries
    '''
    # initialize variables
    out = []
    csvfile = open(filename,'r',encoding='UTF-8')
    fileToRead = csv.reader(csvfile)
    
    headers = next(fileToRead)
    length=len(headers)
    for i in range(length):
        if headers[i]=='Rating':
            headers[i]='Class'
            
    # iterate through rows of actual data
    for row in fileToRead:
        if len(row) == len(headers) and row[2] != 'NaN':
            out.append(dict(zip(headers, row)))
            
    return out

Parse function in here is used to change the format of the raw data from Kaggle to the into “UTF-8” format which could be used as our training and testing data. And unit_test function maintains the testing process of our model.

In [61]:
import random
import matplotlib.pyplot as plt
import numpy as np

def testID3AndEvaluate():
    data = [dict(a=1, b=0, Class=1), dict(a=1, b=1, Class=1)]
    tree = ID3(data, 0)
    if tree != None:
        ans = evaluate(tree, dict(a=1, b=0))
        if ans != 1:
            print("ID3 test failed.")
        else:
            print("ID3 test succeeded.")
    else:
        print("ID3 test failed -- no tree returned")

def testPruning():
    data = [dict(a=0, b=1, c=1, d=0, Class=1), dict(a=0, b=0, c=1, d=0, Class=0), dict(a=0, b=1, c=0, d=0, Class=1), dict(a=1, b=0, c=1, d=0, Class=0), dict(a=1, b=1, c=0, d=0, Class=0), dict(a=1, b=1, c=0, d=1, Class=0), dict(a=1, b=1, c=1, d=0, Class=0)]
    validationData = [dict(a=0, b=0, c=1, d=0, Class=1), dict(a=1, b=1, c=1, d=1, Class = 0)]
    tree = ID3(data, 0)
    prune(tree, validationData)
    if tree != None:
        ans = evaluate(tree, dict(a=0, b=0, c=1, d=0))
        if ans != 1:
            print("pruning test failed.")
        else:
            print("pruning test succeeded.")
    else:
        print("pruning test failed -- no tree returned.")


def testID3AndTest():
    trainData = [dict(a=1, b=0, c=0, Class=1), dict(a=1, b=1, c=0, Class=1), 
    dict(a=0, b=0, c=0, Class=0), dict(a=0, b=1, c=0, Class=1)]
    testData = [dict(a=1, b=0, c=1, Class=1), dict(a=1, b=1, c=1, Class=1), 
    dict(a=0, b=0, c=1, Class=0), dict(a=0, b=1, c=1, Class=0)]
    tree = ID3(trainData, 0)
    fails = 0
    if tree != None:
        acc = test(tree, trainData)
        if acc == 1.0:
            print("testing on train data succeeded.")
        else:
            print("testing on train data failed.")
            fails = fails + 1
        acc = test(tree, testData)
        if acc == 0.75:
            print("testing on test data succeeded.")
        else:
            print("testing on test data failed.")
            fails = fails + 1
        if fails > 0:
            print("Failures: ", fails)
        else:
            print("testID3AndTest succeeded.")
    else:
        print("testID3andTest failed -- no tree returned.")	

# inFile - string location of the house data file
def testPruningOnHouseData(inFile):
    withPruning = []
    withoutPruning = []
    data = parse(inFile)
    for i in range(100):
        random.shuffle(data)
        train = data[: 9 * len(data)//10]
        test = data[9 * len(data)//10:]

        tree = ID3(train, 'democrat')
        acc = test(tree, train)
        print("training accuracy: ",acc)
        acc = test(tree, test)
        print("test accuracy: ",acc)
  
        acc = test(tree, test)
        print("pruned tree test accuracy: ",acc)
        withPruning.append(acc)

        acc = test(tree, test)
        print("no pruning test accuracy: ",acc)
        withoutPruning.append(acc)
    print(withPruning)
    print(withoutPruning)
    print("average with pruning",sum(withPruning)/len(withPruning)," without: ",sum(withoutPruning)/len(withoutPruning))

def SimilarOnHouseData(data, trainSize):
    res=[]
    withPruning = []
    withoutPruning = []
    trainPartSize=int(trainSize/2)
    validSize=int(3*trainSize/4)
    for i in range(100):
        random.shuffle(data)
        train = data[:trainPartSize]
        valid = data[trainPartSize:validSize]
        test = data[validSize:]

        tree = ID3(train, 'democrat')
        acc = test(tree, train)
        acc = test(tree, valid)
        acc = test(tree, test)

        prune(tree, valid)
        acc = test(tree, train)
        acc = test(tree, valid)
        acc = test(tree, test)
        withPruning.append(acc)

        tree = ID3(train + valid, 'democrat')
        acc = test(tree, test)
        withoutPruning.append(acc)
    print("average with pruning", sum(withPruning) / len(withPruning), " without: ",sum(withoutPruning) / len(withoutPruning))
    res.append(sum(withPruning) / len(withPruning))
    res.append(sum(withoutPruning) / len(withoutPruning))
    return res

def graphtest(infile):
    data = parse(infile)
    pruneRes=[]
    WithoutPruneRes=[]
    for trainSize in range(10,301,10):
        result=SimilarOnHouseData(data,trainSize)
        pruneRes.append(result[0])
        WithoutPruneRes.append(result[1])

    x_axis=np.arange(10,301,10)

    axes = plt.gca()
    axes.set_ylim([0.6, 1])

    plt.figure(1, dpi=50)
    plt.plot(x_axis, pruneRes, color='r',marker='.')
    plt.plot(x_axis, WithoutPruneRes, color='b', marker='.')
    labels=['average with pruning', 'average without pruning']
    plt.legend(labels,loc=4)
    plt.title('average accuracy between different training sizes')
    plt.xlabel('number of training examples')
    plt.ylabel('accuracy on test data')
    plt.show()

In [62]:
import random

def mini_grader():

    data = [dict(a=1, b=0, Class=1), dict(a=1, b=1, Class=1)]

    try:
        tree = ID3(data, 0)
        if tree != None:
            ans = evaluate(tree, dict(a=1, b=0))
            if ans != 1:
                print("ID3 test 1 failed.")
            else:
                print("ID3 test 1 succeeded.")
        else:
            print("ID3 test 1 failed -- no tree returned")
    except Exception:
        print("ID3 test 1 failed runtime error")

    data = [dict(a=1, b=0, Class=0), dict(a=1, b=1, Class=1)]

    try:
        tree = ID3(data, 0)
        if tree != None:
            ans = evaluate(tree, dict(a=1, b=0))
            if ans != 0:
                print("ID3 test 2 failed.")
            else:
                print("ID3 test 2 succeeded.")
        else:
            print("ID3 test 2 failed -- no tree returned")
    except Exception:
        print("ID3 test 2 failed runtime error")

    data = [dict(a=1, b=0, Class=2), dict(a=1, b=1, Class=1),
            dict(a=2, b=0, Class=2), dict(a=2, b=1, Class=3),
            dict(a=3, b=0, Class=1), dict(a=3, b=1, Class=3)]

    try:
        tree = ID3(data, 0)
        if tree != None:
            ans = evaluate(tree, dict(a=1, b=0))
            if ans != 2:
                print("ID3 test 3-1 failed.")
            else:
                print("ID3 test 3-1 succeeded.")
            ans = evaluate(tree, dict(a=1, b=1))
            if ans != 1:
                print("ID3 test 3-2 failed.")
            else:
                print("ID3 test 3-2 succeeded.")
        else:
            print("ID3 test 3 failed -- no tree returned")
    except Exception:
        print("ID3 test 3 failed runtime error")

    data = [dict(a=1, b=0, c='?', Class=1), dict(a=1, b=3, c=2, Class=1),
            dict(a=2, b='?', c=1, Class=2), dict(a=2, b=1, c=3, Class=2),
            dict(a=3, b=0, c=1, Class=3), dict(a=3, b=2, c='?', Class=3)]

    try:
        tree = ID3(data, 0)
        if tree != None:
            ans = evaluate(tree, dict(a=1, b=1, c=1))
            if ans != 1:
                print("ID3 test 4-1 failed.")
            else:
                print("ID3 test 4-1 succeeded.")
            ans = evaluate(tree, dict(a=2, b=0, c=0))
            if ans != 2:
                print("ID3 test 4-2 failed.")
            else:
                print("ID3 test 4-2 succeeded.")
        else:
            print("ID3 test 4 failed -- no tree returned")
    except Exception:
        print("ID3 test 4 failed runtime error")

if __name__ == "__main__":
    testPruningOnHouseData('googleplaystore.csv')

<h2>Results</h2>

training accuracy:  0.9988137603795967
<br>test accuracy:  0.9787234042553191

<h2>Future works</h2>

For our future work. First, the data preprocessing needs much more effort. Right now, there are some examples in the same attribute with different value but have same meaning, we need to deal with this case first. Meanwhile, we must make sure that every type of attribute is number but not string, the numeralization of the attributes is needed here, one-hot encoding may used here to handle this problem.

Then, though our decision tree gets a good result, but we don’t think that is perfect. Obviouly, some attributes like “Last Updated” might be useless. We will try to narrow down the attributes we used. After that, we will test our examples with several different method and different partition of data, try to understand why some of them are good, but others are not. In the end, we will try to come up with a criteria which take the most influential attributes into consideration for the app developers for their better development. 